<a href="https://colab.research.google.com/github/Gilgamesh60/AnimeCharactersFaceRecognition-unfinished-/blob/main/Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms

In [ ]:
path_test = '/content/drive/MyDrive/cropped_images'

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [ ]:
CATEGORIES=['ayanokoji_kiyotaka','rikka_takanashi','nami','emilia_re-zero','rimuru','senjogahara_hitagi','miku_nakano','lelouch_lamperouge','hayasaka_ai','marin_kitagawa']

In [ ]:
class_dict={}
count=0
for category in CATEGORIES:
  class_dict[category]=count
  count+=1
class_dict

{'ayanokoji_kiyotaka': 0,
 'rikka_takanashi': 1,
 'nami': 2,
 'emilia_re-zero': 3,
 'rimuru': 4,
 'senjogahara_hitagi': 5,
 'miku_nakano': 6,
 'lelouch_lamperouge': 7,
 'hayasaka_ai': 8,
 'marin_kitagawa': 9}

In [ ]:
images = []
y=[]
IMG_SIZE =227
def createTrainingData():
  for category in CATEGORIES:
    path = os.path.join(path_test, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img))
      if img_array is None:
        continue
      new=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
      coloured=cv2.cvtColor(new,cv2.COLOR_BGR2RGB)
      images.append(coloured)
      y.append(class_dict[category])

In [ ]:
createTrainingData()

In [ ]:
X =[]
y =[]
for features, label in images:
  X.append(features)
  y.append(label)

In [ ]:
IMG_SIZE = 200

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

y = np.array(y)

In [ ]:
import os
root_dir='/content/drive/MyDrive/cropped_images/'
os.makedirs(root_dir+'train/')
os.makedirs(root_dir+'test/')

In [ ]:
import numpy as np
import shutil
import random
test_ratio=0.2
CATEGORIES=['ayanokoji_kiyotaka','rikka_takanashi','nami','emilia_re-zero','rimuru','senjogahara_hitagi','miku_nakano','lelouch_lamperouge','hayasaka_ai','marin_kitagawa']
for category in CATEGORIES:
  os.makedirs(root_dir+'train/'+category)
  os.makedirs(root_dir+'test/'+category)
  train_files=[]
  test_files=[]
  src=root_dir+category
  files=os.listdir(src)
  np.random.shuffle(files)
  train_images, test_images = np.split(np.array(files),[int(len(files)* (1 - test_ratio))])
  train_files=[src+'/'+name for name in train_images]
  test_files=[src+'/'+name for name in train_images]
  for file_name in train_files:
    shutil.copy(file_name,root_dir+'train/'+category)
  for file_name in test_files:
    shutil.copy(file_name,root_dir+'test/'+category)

In [ ]:
transformer=transforms.Compose([transforms.Resize(32),transforms.ToTensor()])

In [ ]:
train_dataset=datasets.ImageFolder('/content/drive/MyDrive/cropped_images/train',transform=transformer)
test_dataset=datasets.ImageFolder('/content/drive/MyDrive/cropped_images/test',transform=transformer)

In [ ]:
train_set=torch.utils.data.DataLoader(train_dataset,shuffle=True,batch_size=4)
test_set=torch.utils.data.DataLoader(test_dataset,shuffle=True,batch_size=4)

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size=7, stride=2,padding=2)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc1  = nn.Linear(in_features= 256*3*3, out_features= 1024)
        self.fc2  = nn.Linear(in_features= 1024, out_features= 512)
        self.fc3 = nn.Linear(in_features=512 , out_features=10)


    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model=AlexNet()
model.train()

AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(2, 2))
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2304, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)

In [ ]:
from torch.nn.modules.container import ModuleList
from torch import optim
#training the model
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=AlexNet()
model=model.to(device=device)

learning_rate=1e-4
load_model=True
epochs=17
loss_function = nn.CrossEntropyLoss()
optimizer =optim.Adam(params= model.parameters(),lr=learning_rate)
for epoch in range(epochs):
    
    for image,label in train_set:
       image=image.to(device)
       label=label.to(device)
       output = model(image)
       loss = loss_function(output,label)

       optimizer.zero_grad()
        
       loss.backward()

       optimizer.step()
    
    print(f'Epoch {epoch+1}/{epochs}: Training Loss: {loss.item()}')

Epoch 1/17: Training Loss: 2.2955541610717773
Epoch 2/17: Training Loss: 2.9832160472869873
Epoch 3/17: Training Loss: 1.4030698537826538
Epoch 4/17: Training Loss: 3.3637428283691406
Epoch 5/17: Training Loss: 1.1573679447174072
Epoch 6/17: Training Loss: 0.7340166568756104
Epoch 7/17: Training Loss: 0.22163547575473785
Epoch 8/17: Training Loss: 0.10277438163757324
Epoch 9/17: Training Loss: 0.0014359832275658846
Epoch 10/17: Training Loss: 0.5269375443458557
Epoch 11/17: Training Loss: 0.017077535390853882
Epoch 12/17: Training Loss: 0.5937560796737671
Epoch 13/17: Training Loss: 0.0019223210401833057
Epoch 14/17: Training Loss: 0.020727848634123802
Epoch 15/17: Training Loss: 0.005889357533305883
Epoch 16/17: Training Loss: 0.19996114075183868
Epoch 17/17: Training Loss: 0.0007490348070859909


In [ ]:
correct=0
incorrect=0

with torch.no_grad():
  for image,label in test_set:
      image = image.to(device)
      label = label.to(device)

      output = model(image)
      temp,pred=torch.max(output,1)
      correct+= (pred==label).sum().item()
      incorrect+= (pred!=label).sum().item()

      model_accuracy =(correct/(correct+incorrect))*100
  
  
  
print("The model accuracy is :",model_accuracy,"%",sep=" ")

The model accuracy is : 96.86520376175548 %
